In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image
from io import BytesIO

# Task 1: Web Scraping
url = "https://pokemondb.net/pokedex/all"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Locate the table
table = soup.find("table", class_="data-table")
rows = table.find_all("tr")

# Extract data
columns = [col.text for col in rows[0].find_all("th")]
data = []

for row in rows[1:]:
    cells = row.find_all("td")
    row_data = [cell.text.strip() for cell in cells]
    # Add sprite image URL
    sprite_url = row.find("img")["src"]
    row_data.append(sprite_url)
    data.append(row_data)

columns.append("Sprite")
df = pd.DataFrame(data, columns=columns)

# Save the data
df.to_csv("pokemon_data.csv", index=False)

# Bonus: Display sprite images
def display_sprite(pokemon_name):
    row = df[df["Name"].str.contains(pokemon_name, case=False, na=False)]
    if not row.empty:
        img_url = row.iloc[0]["Sprite"]
        response = requests.get(img_url)
        img = Image.open(BytesIO(response.content))
        img.show()
    else:
        print("Pokemon not found!")

# Task 2: Exploratory Data Analysis
# Convert stats columns to numeric for analysis
stats_columns = ["HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed", "Total"]
for col in stats_columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# a. Strongest Pokémon of each type
def strongest_per_type():
    types = df["Type"].unique()
    strongest = {}
    for t in types:
        type_df = df[df["Type"].str.contains(t, na=False)]
        if not type_df.empty:
            strongest[t] = type_df.loc[type_df["Total"].idxmax()]
    return strongest

# b. Best attackers
def best_attackers():
    return df.sort_values("Attack", ascending=False).head(10)

# c. Average stats per type
def average_stats_per_type():
    return df.groupby("Type")[stats_columns].mean()

# Task 3: BONUS - GUI Application (Simple CLI version for now)
def pokedex():
    while True:
        print("\nPokedex Options:")
        print("1. Search for a Pokémon")
        print("2. Show strongest Pokémon by type")
        print("3. Show best attackers")
        print("4. Show average stats by type")
        print("5. Exit")
        choice = input("Enter your choice: ")

        if choice == "1":
            name = input("Enter Pokémon name: ")
            display_sprite(name)
        elif choice == "2":
            strongest = strongest_per_type()
            for t, data in strongest.items():
                print(f"Type: {t}, Strongest: {data['Name']} with Total Stats: {data['Total']}")
        elif choice == "3":
            attackers = best_attackers()
            print(attackers[["Name", "Attack"]])
        elif choice == "4":
            averages = average_stats_per_type()
            print(averages)
        elif choice == "5":
            break
        else:
            print("Invalid choice, please try again.")

# Run the Pokedex CLI
if __name__ == "__main__":
    pokedex()



Pokedex Options:
1. Search for a Pokémon
2. Show strongest Pokémon by type
3. Show best attackers
4. Show average stats by type
5. Exit
Enter your choice: 2
Type: Grass Poison, Strongest: Venusaur Mega Venusaur with Total Stats: 625
Type: Fire, Strongest: Groudon Primal Groudon with Total Stats: 770
Type: Fire Flying, Strongest: Ho-oh with Total Stats: 680
Type: Fire Dragon, Strongest: Charizard Mega Charizard X with Total Stats: 634
Type: Water, Strongest: Kyogre Primal Kyogre with Total Stats: 770
Type: Bug, Strongest: Pinsir Mega Pinsir with Total Stats: 600
Type: Bug Flying, Strongest: Pinsir Mega Pinsir with Total Stats: 600
Type: Bug Poison, Strongest: Beedrill Mega Beedrill with Total Stats: 495
Type: Normal Flying, Strongest: Pidgeot Mega Pidgeot with Total Stats: 579
Type: Normal, Strongest: Arceus with Total Stats: 720
Type: Dark Normal, Strongest: Obstagoon with Total Stats: 520
Type: Poison, Strongest: Eternatus Eternamax with Total Stats: 1125
Type: Electric, Strongest: Z

Enter your choice: 3
                         Name  Attack
201      Mewtwo Mega Mewtwo X     190
274  Heracross Mega Heracross     185
956                   Kartana     181
475    Rayquaza Mega Rayquaza     180
478       Deoxys Attack Forme     180
473    Groudon Primal Groudon     180
545    Garchomp Mega Garchomp     170
777       Kyurem Black Kyurem     170
961   Necrozma Ultra Necrozma     167
578      Gallade Mega Gallade     165

Pokedex Options:
1. Search for a Pokémon
2. Show strongest Pokémon by type
3. Show best attackers
4. Show average stats by type
5. Exit
Enter your choice: 4
                      HP      Attack     Defense     Sp. Atk     Sp. Def  \
Type                                                                       
Bug            48.920000   48.480000   54.960000   37.560000   46.040000   
Bug Dark       71.000000  102.000000   78.000000   52.000000   55.000000   
Bug Electric   63.500000   69.000000   73.750000   88.500000   65.000000   
Bug Fairy      50.00000